In [ ]:
# Create Stash app client

import dotenv

from libraries.client_stashapp import get_stashapp_client


dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
import base64
import subprocess
from pathlib import Path


def convert_to_data_url(file_path):
    with Path(file_path).open("rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
        file_extension = Path(file_path).suffix[1:]  # Remove the dot
        return f"data:video/{file_extension};base64,{encoded_string}"

def update_tag_image(file_path):
    # Get the file name without extension
    tag_name = Path(file_path).stem

    # Convert the image to a data URL
    data_url = convert_to_data_url(file_path)

    # Find the tag by name
    tag = stash.find_tag(tag_name)
    if not tag:
        print(f"Tag not found: {tag_name}")
        return

    has_an_image = False # "default=" not in tag.get("image_path", "").split("?", 1)[-1]

    if has_an_image:
        return

    # Update the tag's image
    result = stash.update_tag({
        "id": tag["id"],
        "image": data_url
    })
    print(f"Updated image for tag: {tag_name}")

def convert_mov_to_webm(input_file, output_file):
    ffmpeg_path = "ffmpeg"
    command = [
        ffmpeg_path,
        "-i", input_file,
        "-c:v", "libvpx-vp9",
        "-crf", "30",
        "-b:v", "0",
        "-an",
        output_file
    ]
    subprocess.run(command, check=True)

# Directory containing .mov files
mov_dir_path = Path(r"X:\Stash Tags")

# Convert .mov files to .webm
for file_entry in mov_dir_path.iterdir():
    if file_entry.suffix.lower() == ".mov":
        webm_file = file_entry.with_suffix(".webm")

        if not webm_file.exists():
            print(f"Converting {file_entry.name} to {webm_file.name}...")
            convert_mov_to_webm(str(file_entry), str(webm_file))

print("Conversion process complete.")

# Update tag images
for file_entry in mov_dir_path.iterdir():
    if file_entry.suffix.lower() == ".webm":
        update_tag_image(str(file_entry))